In [76]:
import pandas as pd

In [77]:
articles = pd.read_csv('/Users/calebharding/Documents/BYU/2023-2024/China_Project/articles/scrape_articles/get_qiushi_articles/all_qiushi_articles.csv')
print(f"Total number of articles scraped: {len(articles)}")

Total number of articles scraped: 2181


In [78]:
missing_year = articles[articles['year'].isnull()]
# missing_year.to_csv('missing_year.csv')
len(missing_year)

0

In [79]:
def determine_edition_quarter(edition_number):
    if edition_number in range(1,7):
        return 1
    elif edition_number in range(7,13):
        return 2
    elif edition_number in range(13, 19):
        return 3
    else:
        return 4
    
def combine_year_quarter(row):
    return str(row["year"]) + " Q" + str(row["quarter"])

# drop articles without a year
articles = articles.dropna(subset=["year"])
print(f"Number of articles after dropping missing years: {len(articles)}")
    
# add a column for the quarter
articles["quarter"] = articles['edition'].map(lambda x: determine_edition_quarter(x))

# add a column for the year/quarter combo
articles = articles.astype({"year": int})
articles["year_quarter"] = articles.apply(combine_year_quarter, axis=1)

Number of articles after dropping missing years: 2181


In [80]:
# verify that the textless articles are what we think - statistical reports, not just
# articles that failed to scrape
textless_articles = articles[articles['text'].isnull()]
textless_articles.to_csv('textless_articles.csv')

articles.dropna(subset=['text'], inplace=True)
print(f'Number of articles after dropping those with no text: {len(articles)}')

Number of articles after dropping those with no text: 2058


In [81]:
# somehow some duplicates of Xi Jinping's speeches got in there... cause their website is dumb. They have duplicate links though

# go through all the articles, add their link to a set, if the link is already in the set, drop it
link_set = set()

for index, row in articles.iterrows():
    if row['url'] in link_set:
        articles.drop(index, inplace=True)
    else:
        link_set.add(row['url'])

print(f'Number of articles after dropping duplicate links: {len(articles)}')

Number of articles after dropping duplicate links: 2042


In [82]:
# drop articles that are top-level articles with duplicate text contained - they have three+ articles
# from the same edition copied inside of them
suspected_toplevel = articles[articles.text.str.contains('编者按')]
suspected_toplevel['url'].to_csv('suspected_toplevel.csv')
toplevel_links = list(suspected_toplevel['url'])
print(f'Suspected top-level links: {len(toplevel_links)}')

# remove the few links that are actually legit (determined by looking at all 16)
legit_links = ['http://www.qstheory.cn/dukan/qs/2019-01/16/c_1123987212.htm',
               'http://www.qstheory.cn/dukan/qs/2019-01/16/c_1123987342.htm',
               'http://www.qstheory.cn/dukan/qs/2019-07/01/c_1124690404.htm']
for link in legit_links:
    toplevel_links.remove(link)
print(f'Top-level links: {len(toplevel_links)}')

Suspected top-level links: 16
Top-level links: 13


In [83]:
# drop the articles
articles = articles[~articles['url'].isin(toplevel_links)]
print(f'Number of articles after dropping top-level entries: {len(articles)}')

Number of articles after dropping top-level entries: 2029


In [84]:
# for the sake of consistency, drop the articles that are from 2024, since there 
# isn't a complete quarter yet
articles = articles[articles['year'] != 2024]
print(f'Number of articles after dropping 2024 entries: {len(articles)}')

Number of articles after dropping 2024 entries: 1998


In [85]:
# sort it by date
articles.sort_values(by='date', inplace=True)

In [86]:
articles.head(5)

,authors,title,date,year,edition,url,text,quarter,year_quarter
1790,['习近平'],辩证唯物主义是中国共产党人的世界观和方法论,2018-12-31,2019,1.0,http://www.qstheory.cn/dukan/qs/2018-12/31/c_1...,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日，纪念马克思诞辰200周年...,1,2019 Q1
1799,"['刘雅鸣', '陈聪', '李亚楠', '宋晓东']",兰考：会它千顷澄碧,2019-01-01,2019,1.0,http://www.qstheory.cn/dukan/qs/2019-01/01/c_1...,焦裕禄当年亲手栽下的幼桐已长成大树，人们亲切地叫它“焦桐”。 新华社记者 冯大鹏/摄兰考谷营...,1,2019 Q1
1804,['国家统计局'],经济社会发展统计：改革开放40年辉煌成就（经济篇）,2019-01-01,2019,1.0,http://www.qstheory.cn/dukan/qs/2019-01/01/c_1...,我国国民经济主要指标注：国内生产总值增速按可比价计算，2017年国内生产总值总量不等于各产业...,1,2019 Q1
1803,['《中直党建》评论员'],多敲警钟才能少敲丧钟,2019-01-01,2019,1.0,http://www.qstheory.cn/dukan/qs/2019-01/01/c_1...,中央和国家机关警示教育大会，对各部门开展党风廉政警示教育活动进行了部署。落实好会议精神，把警...,1,2019 Q1
1802,['李君'],学理论是党员领导干部的责任,2019-01-01,2019,1.0,http://www.qstheory.cn/dukan/qs/2019-01/01/c_1...,编辑同志：我是广西南宁的一名基层理论工作者，从事理论工作近20年了。在长期工作实践中，我深深...,1,2019 Q1


In [87]:
articles.isna().sum()

authors         289
title             0
date              0
year              0
edition           0
url               0
text              0
quarter           0
year_quarter      0
dtype: int64

In [88]:
# convert the authors to a string format so it matches the ASX format
# when pulling from a CSV, pandas does not store the cell as a list, but as a string

from ast import literal_eval

# fill the missing values so they don't throw errors
articles['authors'] = articles['authors'].fillna("")

from ast import literal_eval

for index, row in articles.iterrows():
    if row['authors'] != "":
        author_list = literal_eval(row['authors'])
        authors = " ".join(author_list)
        row['authors'] = authors

        articles.loc[index] = row


In [90]:
articles.head(5)

,authors,title,date,year,edition,url,text,quarter,year_quarter
1790,习近平,辩证唯物主义是中国共产党人的世界观和方法论,2018-12-31,2019,1.0,http://www.qstheory.cn/dukan/qs/2018-12/31/c_1...,辩证唯物主义是中国共产党人的世界观和方法论习近平2018年5月4日，纪念马克思诞辰200周年...,1,2019 Q1
1799,刘雅鸣 陈聪 李亚楠 宋晓东,兰考：会它千顷澄碧,2019-01-01,2019,1.0,http://www.qstheory.cn/dukan/qs/2019-01/01/c_1...,焦裕禄当年亲手栽下的幼桐已长成大树，人们亲切地叫它“焦桐”。 新华社记者 冯大鹏/摄兰考谷营...,1,2019 Q1
1804,国家统计局,经济社会发展统计：改革开放40年辉煌成就（经济篇）,2019-01-01,2019,1.0,http://www.qstheory.cn/dukan/qs/2019-01/01/c_1...,我国国民经济主要指标注：国内生产总值增速按可比价计算，2017年国内生产总值总量不等于各产业...,1,2019 Q1
1803,《中直党建》评论员,多敲警钟才能少敲丧钟,2019-01-01,2019,1.0,http://www.qstheory.cn/dukan/qs/2019-01/01/c_1...,中央和国家机关警示教育大会，对各部门开展党风廉政警示教育活动进行了部署。落实好会议精神，把警...,1,2019 Q1
1802,李君,学理论是党员领导干部的责任,2019-01-01,2019,1.0,http://www.qstheory.cn/dukan/qs/2019-01/01/c_1...,编辑同志：我是广西南宁的一名基层理论工作者，从事理论工作近20年了。在长期工作实践中，我深深...,1,2019 Q1


In [92]:
articles.to_csv('validated_qiushi_articles.csv', index=False)